# Predict TripAdvisor Rating


# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
df_cities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')


In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
df_cities

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


In [ ]:
df_cities = df_cities.rename(columns = {'city': 'City'})
df_cities = df_cities[(df_cities.City.isin(data.City.values)) & (df_cities.capital == 'primary')].sort_values(by = 'City')
df_cities = df_cities[['City','population']]
data = data.merge(df_cities, how = 'left', on = 'City')


In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

# Cleaning and Prepping Data


## 1. Обработка NAN 


In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

#удалить, с таким признаком результат хуже
#mean_reviews_in_city = data.groupby('City').mean()['Number of Reviews']
#data['Number of Reviews'] = data.apply(lambda x: mean_reviews_in_city.loc[x['City']] if pd.isna(x['Number of Reviews']) else x['Number of Reviews'],axis = 1)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

In [ ]:
#удалить, с таким признаком результат хуже

#нормализуем количество ресторанов по городам

#from sklearn import preprocessing
#restaurants_in_city = data.groupby('City').count()['Restaurant_id']
#data['restaurants_in_city_norm'] = preprocessing.normalize(data.apply(lambda x: restaurants_in_city.loc[x['City']], axis = 1).values.reshape(-1, 1))
#data['restaurants_in_city_norm']

In [ ]:
#добавим дополнительные признаки для городов

data['population_na'] = pd.isna(data.population).astype('uint8')
data['population'].fillna(0)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'
data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x =='$' else 2 if x =='$$ - $$$' else 3 if x == '$$$$' else 0)


In [ ]:
data.info()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

In [ ]:
#удалить, с таким признаком результат хуже
#data = pd.get_dummies(data, columns = ['Price Range'], dummy_na=True)

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# .....

#обработаем виды кухонь, приводим строку в типу list
pattern = re.compile('\w+\s*\w*')
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: pattern.findall(x) if pd.isna(x)==False else [])

#закодируем в dummies
cuisines_dummies = pd.get_dummies(data['Cuisine Style'].explode()).groupby(level = 0).sum()
data = pd.concat([data,cuisines_dummies], axis = 1)


In [ ]:
#удалить, с таким признаком результат хуже
#попробуем закодировать только топ кухонь(например, 3,5,10 или 20)
#top5 = data['Cuisine Style'].explode().value_counts().head(5).index.to_list()
#cuisines_dummies = pd.get_dummies(data['Cuisine Style'].explode()).groupby(level = 0).sum()[top5]
#data = pd.concat([data,cuisines_dummies], axis = 1)

In [ ]:
#удалить, с таким признаком результат хуже

#добавим признак с количеством ресторанов в сети
#number_of_restaurants = data.groupby('Restaurant_id').count()['Cuisine Style']
#data['number_of_restaurants'] = data['Restaurant_id'].apply(lambda x: number_of_restaurants.loc[x])


In [ ]:
#удалить, с таким признаком результат хуже

#посмотрми разницу в днях между первым и последним отзывом

#pattern2 = re.compile('[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]')

#def make_datelist(string_review):
#    datelist = []
#    for i in pattern2.findall(string_review):
#        datelist.append(datetime.strptime(i, '%m/%d/%Y'))
#    return datelist

#data['Reviews_dates'] = data.Reviews.fillna('').apply(lambda x: make_datelist(x))
#data['Reviews_dates_dif'] = data.apply(lambda x: (max(x['Reviews_dates']) - min(x['Reviews_dates'])).days if len(x['Reviews_dates'])==2 else 0,axis=1)



In [ ]:
data.info()

# EDA 

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

In [ ]:
#data.corr()[((data.corr() > 0.05) & (data.corr() < 0.9)) | (data.corr() < -0.05) & (data.corr() > -0.9)]

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

df_cities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')
df_cities = df_cities.rename(columns = {'city': 'City'})
df_cities = df_cities[(df_cities.City.isin(data.City.values)) & (df_cities.capital == 'primary')].sort_values(by = 'City')
df_cities = df_cities[['City','country','population']]
data = data.merge(df_cities, how = 'left', on = 'City')

data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')

    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    # ################### 3. Encoding ############################################################## 

    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....    
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: 1 if x =='$' else 2 if x =='$$ - $$$' else 3 if x == '$$$$' else 0)

    pattern = re.compile('\w+\s*\w*')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: pattern.findall(x) if pd.isna(x)==False else [])
    cuisines_dummies = pd.get_dummies(df_output['Cuisine Style'].explode()).groupby(level = 0).sum()
    df_output = pd.concat([df_output,cuisines_dummies], axis = 1)
    

    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ...
    df_output['population_na'] = pd.isna(df_output.population).astype('uint8')
    df_output['population'].fillna(0, inplace=True)     


    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
df_preproc

In [ ]:
#удалить, результат ухудшился
#попробуем использовать только самые важные признаки
#df_preproc = df_preproc[['sample','Rating','City_Amsterdam', 'City_Berlin', 'City_Prague', 'City_Milan',
#        'City_Madrid', 'Price Range', 'City_Barcelona', 'population',
#        'Number of Reviews', 'Ranking']]


In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
#определим самые важные признаки
#len(pd.Series(model.feature_importances_, index=X.columns)), \
#pd.Series(model.feature_importances_, index=X.columns).sort_values()[-30:].index.to_list


# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)